# InfluxDB2.0 Primer

This notebook shows you how to write a dataframe into an Influx instance. Keep all your secret vars in a locally stored `.env` file

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS
from dotenv import find_dotenv, load_dotenv
# You can generate a Token from the "Tokens Tab" in the UI
load_dotenv(find_dotenv())

True

Code below populates InfluxDB bucket with air quality data in csv format pulled from https://aqicn.org/data-platform/covid19/

For testing purposes you can choose only some city (cities) from a list.

# Set up InfluxDB bindings 

Documentation on Python bindings with examples can be found here:
https://github.com/influxdata/influxdb-client-python

In [2]:
influx = InfluxDBClient(url=os.environ['INFLUX_HOST'], token=os.environ['INFLUX_TOKEN'])
write_api = influx.write_api(write_options=SYNCHRONOUS)

In [3]:
# Read daily data
df = pd.read_csv('https://raw.githubusercontent.com/AntonBiryukovUofC/air_quality_check/luis-testing/src/data/waqi-covid19-airqualitydata-filtered.csv',
                 parse_dates = ['Date']).set_index('Date')
df['year'] = pd.DatetimeIndex(df.index).year
df['month'] = pd.DatetimeIndex(df.index).month
df['DOY'] = pd.DatetimeIndex(df.index).dayofyear

In [4]:
print(df['City'].unique())

['Jieyang' 'Beijing' 'Kunming' 'Hangzhou' 'Chongqing' 'Qingdao' 'Haikou'
 'Qiqihar' 'Guiyang' 'Ürümqi' 'Shenzhen' 'Yunfu' 'Xuchang' 'Yinchuan'
 'Shenyang' 'Lhasa' 'Shanghai' 'Changchun' 'Foshan' 'Nanning' 'Fushun'
 'Hefei' 'Chengdu' 'Hohhot' 'Qinhuangdao' 'Shijiazhuang' 'Shantou'
 'Zhengzhou' 'Nanjing' 'Xining' 'Xi’an' 'Zhuzhou' 'Wuhan' 'Tianjin'
 'Changzhou' 'Nanchang' 'Shiyan' 'Harbin' 'Xinxiang' 'Suzhou' 'Lanzhou'
 'Jinan' 'Changsha' 'Hegang' 'Anyang' 'Taiyuan' 'Guangzhou' 'Fuzhou'
 'Wuxi' 'Ningbo' 'Xiamen' 'Dongguan' 'Hamilton' 'Calgary' 'Winnipeg'
 'Halifax' 'Kitchener' 'Edmonton' 'Mississauga' 'Surrey' 'Québec'
 'Vancouver' 'Victoria' 'Montréal' 'Toronto' 'Ottawa' 'London']


In [ ]:
# Create annual baseline vs 2020 statistics
bins = [2014, 2019, 2020]
labels = ['baseline', '2020']
date_bins = pd.cut(df['year'], bins=bins, labels=labels, include_lowest=True)

annual_stats = df.fillna(0).groupby(['City', date_bins]).agg(
                            {'Country': ['first'],
                             'aqi': ['mean'],
                             'co': ['mean'],
                             'dew': ['mean'],
                             'humidity': ['mean'],
                             'mepaqi': ['mean'],
                             'no2': ['mean'],
                             'o3': ['mean'],
                             'pm10': ['mean'],
                             'pm25': ['mean'],
                             'precipitation': ['mean'],
                             'pressure': ['mean'],
                             'so2': ['mean'],
                             'temperature': ['mean'],
                            }
    ).reset_index()
annual_stats.columns = ['City', 'date_bins','Country', 'aqi', 'co', 'dew', 'humidity', 'mepaqi', 'no2', 'o3', 'pm10', 'pm25', 'preciptation', 'pressure', 'so2', 'temperature']
# annual_stats.head(10)
annual_stats = annual_stats.pivot(index=['City','Country'], columns='date_bins').swaplevel(0, 1, axis=1).sort_index(axis=1)
# annual_stats.head(10)
# annual_stats['2020'].head(10)
annual_stats_change = (annual_stats['2020'].divide(annual_stats['baseline']) - 1) * 100
annual_stats_change = annual_stats_change.round(1)
# annual_stats_change.head(10)

In [ ]:
# Read cities lat, long and other general information
# source: https://simplemaps.com/data/world-cities
cities = pd.read_csv('..\src\data\worldcities.csv', usecols = ['city','iso2', 'lat','lng'])

# There are some cities within China with same name in different provinces (e.g. Changsha)
# AQ data does not have province information to resolve
# for now decided to take coordinates of first city in database, which has largest population
cities = cities.drop_duplicates(subset=['city', 'iso2'])

cities= cities.rename(columns={"city": "City", "iso2": "Country"}).set_index(['City','Country'])
cities.head(10)
cities.loc['Calgary']

In [ ]:
# Merge annual stats with cities lat-longs so they can be displayed on map
result = pd.merge(annual_stats_change, cities, how="left", left_index=True, right_index=True)
# result.head(10)

# some cities do not have coordinates in current cities file used
# for now those cities are dropped
result.dropna(subset=['lat', 'lng'],inplace=True)
result.head(10)

In [ ]:
# Write annual merged cities file with stats to a csv
# the csv file is later read by Webapp to create map
result.to_csv('..\src\data\city_stats.csv')

In [ ]:
# Create monthly baseline vs 2020 statistics
bins = [2014, 2019, 2020]
labels = ['baseline', '2020']
date_bins = pd.cut(df['year'], bins=bins, labels=labels, include_lowest=True)

annual_stats = df.fillna(0).groupby(['City', 'month', date_bins]).agg(
                            {'Country': ['first'],
                             'aqi': ['mean'],
                             'co': ['mean'],
                             'dew': ['mean'],
                             'humidity': ['mean'],
                             'mepaqi': ['mean'],
                             'no2': ['mean'],
                             'o3': ['mean'],
                             'pm10': ['mean'],
                             'pm25': ['mean'],
                             'precipitation': ['mean'],
                             'pressure': ['mean'],
                             'so2': ['mean'],
                             'temperature': ['mean'],
                            }
    ).reset_index()
annual_stats.columns = ['City', 'month', 'date_bins','Country', 'aqi', 'co', 'dew', 'humidity', 'mepaqi', 'no2', 'o3', 'pm10', 'pm25', 'preciptation', 'pressure', 'so2', 'temperature']
# annual_stats.head(10)
annual_stats = annual_stats.pivot(index=['City','Country', 'month'], columns='date_bins').swaplevel(0, 1, axis=1).sort_index(axis=1)
# annual_stats.head(10)
# annual_stats['2020'].head(10)
annual_stats_change = (annual_stats['2020'].divide(annual_stats['baseline']) - 1) * 100
annual_stats_change = annual_stats_change.round(1)
annual_stats_change = annual_stats_change.replace([np.inf, -np.inf], np.nan)
annual_stats_change.head(10)

In [ ]:
# Write monthly stats file to a csv
# the csv file is later read by Webapp to create map
annual_stats_change.to_csv('..\src\data\city_stats_month.csv')

In [5]:
# InfluxDB free version only allows timestamps younger than 30 days
# Data is shifted to a future date in order to be retained in DB
timeshift = float(2000)
df = df.shift(periods=timeshift, freq="D")
df.head(10)

,Country,City,aqi,co,dew,humidity,mepaqi,no2,o3,pm10,pm25,precipitation,pressure,so2,temperature,year,month,DOY
Date,,,,,,,,,,,,,,,,,,
2024-10-24,CN,Jieyang,NaN,5.8,13.0,53.8,NaN,11.0,40.7,54.0,63.0,99.0,1015.0,6.1,22.0,2019,5,124
2024-11-08,CN,Jieyang,NaN,7.3,27.2,100.0,NaN,5.1,21.2,34.0,63.0,99.2,1007.0,3.6,29.0,2019,5,139
2024-11-18,CN,Jieyang,NaN,6.4,21.0,94.0,NaN,6.9,35.4,30.0,55.0,99.1,1011.0,3.1,23.0,2019,5,149
2024-11-22,CN,Jieyang,NaN,6.8,26.0,100.0,NaN,6.9,17.1,20.0,50.0,NaN,1006.0,3.1,27.0,2019,6,153
2024-10-01,CN,Jieyang,NaN,8.3,21.0,98.8,NaN,9.2,22.0,35.0,68.0,99.8,1011.0,3.6,21.0,2019,4,101
2024-11-08,CN,Beijing,NaN,3.7,0.0,26.5,NaN,7.8,28.5,50.0,85.0,1.5,1007.0,1.6,18.5,2019,5,139
2024-11-12,CN,Beijing,NaN,6.4,10.5,37.5,NaN,25.6,52.6,59.0,85.0,NaN,1003.0,2.6,26.5,2019,5,143
2024-10-16,CN,Beijing,NaN,2.8,4.5,47.0,NaN,9.2,32.5,36.0,70.0,0.1,1020.0,1.1,14.5,2019,4,116
2024-10-23,CN,Beijing,NaN,6.4,6.0,32.0,NaN,19.2,39.0,72.0,142.0,NaN,1013.0,4.6,24.0,2019,5,123


In [18]:
# select columns and cities to pull into dataframe

cols = ['no2','o3','co','so2','temperature','City','Country']
# df_to_write = df.loc[(df['Country'] == 'CA' ) | (df['Country'] == 'CN'),cols]
# df_to_write = df.loc[df['Country'] == 'CN',cols]

# add time shift factor as a column in database
df_to_write['timeshift']=timeshift

df_to_write = df.loc[df['City'].isin(['Shanghai', 'Beijing', 'Tianjin', 'Shenzhen', 'Guangzhou', 'Chengdu', 'Chongqing', 'Dongguan', 'Shenyang','Wuhan']),cols]
# df_to_write = df.loc[df['City'].isin(['Ottawa', 'Toronto', 'Montreal']),cols]
df_to_write.sample(10)

,no2,o3,co,so2,temperature,City,Country
Date,,,,,,,
2025-01-27,10.6,19.9,NaN,4.3,30.0,Shenzhen,CN
2026-02-20,8.3,30.5,5.4,2.6,25.0,Chengdu,CN
2025-07-05,22.9,10.6,10.0,3.6,16.0,Guangzhou,CN
2020-10-01,11.9,7.0,8.9,4.1,NaN,Dongguan,CN
2024-07-25,9.2,24.0,9.1,3.6,19.1,Dongguan,CN
2022-08-25,12.8,40.3,6.1,4.1,NaN,Shenzhen,CN
2023-12-22,11.0,13.0,4.6,4.5,27.0,Shanghai,CN
2025-10-20,22.0,31.7,7.5,5.1,26.0,Wuhan,CN
2020-10-06,38.9,19.9,9.0,15.8,NaN,Shanghai,CN


In [19]:
df_to_write.shape

(134737, 7)

In [20]:
# Write dataframe into InfluxDB bucket
write_api.write(os.environ['INFLUX_BUCKET'], os.environ['INFLUX_ORG'], record=df_to_write, data_frame_measurement_name='luis-airquality',data_frame_tag_columns = ['City','Country'])

In [ ]:
# Check database write operation was ok, with a query

query_api = influx.query_api()

In [ ]:
#query= '''
#from(bucket:"spe-project")
#    |> range(start: 2019-01-01T23:30:00Z, stop: 2020-12-31T00:00:00Z)
#    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
#    '''

query= '''
from(bucket:"ts_spe")
        |> range(start: 2019-01-01T23:30:00Z, stop: 2020-12-31T00:00:00Z)
        |> filter(fn: (r) => r["_measurement"] == "luis-airquality")
        |> group(columns:["City"])
        |> distinct(column:"City")
        |> keep(columns: ["_value"])
        '''

# query= '''
# from(bucket:"ts_spe")
#     |> range(start: 2019-01-01T23:30:00Z, stop: 2020-12-31T00:00:00Z)
#     '''

In [ ]:
test = query_api.query_data_frame(org=os.environ['INFLUX_ORG'], query=query)
# display(test.head())
display(test)

In [10]:
# Code to delete measurement from db
start = "2020-01-01T00:00:00Z"
stop = "2041-01-01T00:00:00Z"
delete_api = influx.delete_api()
delete_api.delete(
    start, 
    stop, 
    '_measurement="luis-airquality"',
    bucket=os.environ['INFLUX_BUCKET'],
    org=os.environ['INFLUX_ORG'],
)